## 并行化步骤

RunnableParallel（又名 RunnableMap）可以轻松并行执行多个 Runnable，并将这些 Runnable 的输出作为映射返回。

RunnableParallel 对于并行运行独立进程也很有用，因为映射中的每个 Runnable 都是并行执行的。

In [1]:
from langchain_openai import ChatOpenAI, OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.3,
)


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

In [3]:
outlinePromptTemplate = '''主题：{theme}
如果要根据主题写一篇文章，请列出文章的大纲。'''

outlinePrompt = ChatPromptTemplate.from_template(outlinePromptTemplate)
outlinePrompt

ChatPromptTemplate(input_variables=['theme'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['theme'], template='主题：{theme}\n如果要根据主题写一篇文章，请列出文章的大纲。'))])

In [4]:
tipsPromptTemplate = '''主题：{theme}
如果要根据主题写一篇文章，应该需要注意哪些方面，才能把这篇文章写好。
'''

tipsPrompt = ChatPromptTemplate.from_template(tipsPromptTemplate)
tipsPrompt

ChatPromptTemplate(input_variables=['theme'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['theme'], template='主题：{theme}\n如果要根据主题写一篇文章，应该需要注意哪些方面，才能把这篇文章写好。\n'))])

In [5]:
query = "2024年中国经济走向与运行趋势"

In [6]:
from langchain_core.output_parsers import StrOutputParser
strParser = StrOutputParser()

In [11]:
%%time
outlineChain = outlinePrompt | model | strParser
outline = outlineChain.invoke({"theme":query})
outline

CPU times: total: 0 ns
Wall time: 7.46 s


'1. 引言：简要介绍中国经济现状和面临的挑战，并说明预测2024年经济走向的重要性。\n2. 第一部分：宏观经济环境分析\n   a. 全球经济形势对中国的潜在影响\n   b. 中国国内政策与经济发展的关系\n3. 第二部分：行业趋势分析\n   a. 制造业和工业发展前景\n   b. 服务业和消费市场趋势\n   c. 科技创新与数字经济的影响\n4. 第三部分：金融领域展望\n   a. 货币政策与利率走势预测\n   b. 资本市场和股票市场的预期\n5. 第四部分：社会因素对经济的影响\n   a. 人口结构变化对劳动力市场和经济发展的影响\n   b. 环境保护政策对产业升级的推动作用\n6. 结论：总结2024年中国经济的潜在趋势，并提出应对挑战的建议。'

In [12]:
%%time
tipsChain = tipsPrompt | model | strParser
tips = tipsChain.invoke({"theme":query})
tips

CPU times: total: 93.8 ms
Wall time: 7.09 s


'要写一篇关于2024年中国经济走向和运行趋势的文章，需要关注以下几个方面：\n\n1. 研究背景：首先，简要介绍当前中国经济发展的现状、面临的挑战以及未来的发展趋势。\n2. 宏观经济分析：从GDP增长、就业率、通货膨胀率、进出口贸易、投资等方面进行分析，预测未来几年中国经济的整体走势。\n3. 行业趋势分析：选取几个具有代表性的行业（如制造业、服务业、金融业等）进行深入分析，探讨这些行业的未来发展趋势和可能面临的问题。\n4. 政策影响：分析政府对经济发展的政策导向，以及这些政策可能对中国经济产生的影响。\n5. 风险与挑战：识别中国经济在未来几年可能面临的风险和挑战，并提出应对措施和建议。\n6. 结论：总结全文，强调中国经济的未来发展趋势，并对中国经济的发展前景进行展望。'

In [9]:
articlePromptTemplate = '''主题：{theme}
大纲：
{outline}

注意事项：
{tips}

请根据上面的主题、大纲和注意事项写出丰富的完整文章内容。
'''

articlePrompt = ChatPromptTemplate.from_template(articlePromptTemplate)
articlePrompt

ChatPromptTemplate(input_variables=['outline', 'theme', 'tips'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['outline', 'theme', 'tips'], template='主题：{theme}\n大纲：\n{outline}\n\n注意事项：\n{tips}\n\n请根据上面的主题、大纲和注意事项写出丰富的完整文章内容。\n'))])

In [13]:
%%time
articleChain = articlePrompt | model | strParser
articleChain.invoke({
    "theme":query,
    "outline":outline,
    "tips":tips
})

CPU times: total: 0 ns
Wall time: 29 s


'引言：\n\n中国经济在过去几十年中取得了显著的增长和发展成就，成为全球第二大经济体。然而，随着全球经济格局的变化以及国内经济结构调整的推进，中国面临着一系列新的挑战。预测2024年中国经济的走向对于制定政策、规划发展方向具有重要意义。本文将从宏观经济环境分析、行业趋势分析、金融领域展望和社会因素影响等方面探讨2024年中国经济的潜在趋势。\n\n第一部分：宏观经济环境分析\n\na. 全球经济形势对中国的潜在影响：\n全球经济的复苏和增长速度将直接影响中国出口贸易和投资活动，进而对中国经济增长产生重要影响。国际贸易摩擦、地缘政治风险以及新兴市场国家的金融波动都可能给中国经济带来不确定性。\n\nb. 中国国内政策与经济发展的关系：\n政府在宏观调控、财政政策和货币政策等方面的决策将对经济发展产生深远影响。例如，中国政府近年来积极推进供给侧结构性改革，旨在提高经济增长的质量和效率，这可能会对2024年的经济增长速度和结构调整产生积极影响。\n\n第二部分：行业趋势分析\n\na. 制造业和工业发展前景：\n随着“中国制造2025”战略的实施，中国制造业正朝着高端化、智能化方向转型升级。预计到2024年，中国制造业将取得显著进步，并在全球价值链中占据更高地位。然而，仍需应对劳动力成本上升、技术壁垒等问题。\n\nb. 服务业和消费市场趋势：\n随着居民收入水平的提高和消费升级的推动，服务业将成为经济增长的重要引擎。预计到2024年，中国服务业占GDP比重将进一步增加，同时，旅游、文化、健康等新兴服务业也将迎来快速发展。\n\nc. 科技创新与数字经济的影响：\n中国在人工智能、5G通信等领域取得了显著进展，这些技术进步将推动数字经济的发展和产业升级。预计到2024年，数字经济的规模将持续扩大，成为经济增长的新动力。然而，数据安全问题、数字鸿沟等挑战仍需应对。\n\n第三部分：金融领域展望\n\na. 货币政策与利率走势预测：\n中国央行将继续保持稳健的货币政策，根据经济形势灵活调整政策工具和力度。预计2024年，利率将维持在相对较低水平，以支持实体经济发展。同时，央行也将加强对金融风险的防范和监管。\n\nb. 资本市场和股票市场的预期：\n随着注册制改革的推进以及投资者结构的优化，中国股市有望迎来更加健康、稳定的发展。然而，仍需关注市场波动、上市公司质量等问题。\

In [15]:
%%time
from langchain_core.runnables import RunnableParallel

map_chain = RunnableParallel(outline=outlineChain, tips=tipsChain)
map_chain.invoke({"theme":query})

CPU times: total: 0 ns
Wall time: 14.5 s


{'outline': '1. 引言：简要介绍中国经济现状和面临的挑战\n2. 第一部分：宏观经济环境分析\n   a. 全球经济形势对中国的冲击\n   b. 中国国内政策调整的影响\n3. 第二部分：行业趋势分析\n   a. 制造业与工业升级\n   b. 数字经济与科技创新\n   c. 消费市场与服务业发展\n4. 第三部分：金融领域展望\n   a. 货币政策和财政政策的预期\n   b. 资本市场与风险管理\n5. 第四部分：区域经济发展\n   a. 京津冀、长三角、粤港澳大湾区等地区的发展机遇\n   b. 中西部地区的崛起与潜力挖掘\n6. 第五部分：结论和建议\n   a. 中国经济面临的挑战和机遇\n   b. 对政府和企业应对策略的建议',
 'tips': '要写好一篇关于2024年中国经济走向和运行趋势的文章，需要从以下几个方面进行考虑：\n\n1. 研究背景：首先需要对当前中国经济的发展状况、政策环境以及国际形势进行分析，为后续的预测提供基础。\n2. 数据收集与分析：通过收集相关数据，包括宏观经济指标、行业数据等，进行深入的分析和解读。\n3. 趋势预测：基于前面的研究和数据分析，对2024年中国经济的走向和运行趋势做出合理的预测。可以采用多种方法，如回归模型、时间序列分析等，以提高预测的准确性。\n4. 政策建议：根据预测结果，提出针对性的政策建议，帮助政府和企业更好地应对未来的经济形势。\n5. 风险评估与对策：对可能影响中国经济发展的因素进行风险评估，并提出相应的应对策略。\n6. 结论和展望：总结全文，并对2024年中国经济的走向和运行趋势做出展望。'}

In [19]:
%%time
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter


map_chain = RunnableParallel(outline=outlineChain, tips=tipsChain, theme=itemgetter("theme"))
map_chain.invoke({"theme":query})

CPU times: total: 0 ns
Wall time: 12.5 s


{'outline': '大纲：\n1. 引言：介绍中国经济在过去几年的发展情况以及全球背景下的挑战和机遇。\n2. 宏观经济指标分析：包括GDP增长、通胀率、失业率等关键指标的预测。\n3. 政策环境与影响：讨论中国政府在2024年的主要政策和目标，并评估其对经济的影响。\n4. 行业趋势与机会：分析不同行业的表现和前景，如制造业、服务业、科技产业等。\n5. 风险因素与挑战：探讨中国经济面临的潜在风险，包括贸易摩擦、债务问题等。\n6. 结论和建议：总结中国经济的预测和发展方向，并提出应对挑战的建议。',
 'tips': '要写好一篇关于2024年中国经济走向与运行趋势的文章，需要从以下几个方面入手：\n\n1. 研究背景：首先需要对当前中国经济形势、政策环境以及国际经济环境进行深入分析，了解影响中国经济的因素和挑战。\n2. 数据收集：通过各种渠道获取最新的经济数据和预测报告，包括GDP增速、通胀率、失业率等指标，以及行业和地区的发展情况。\n3. 趋势预测：根据研究背景和数据收集的结果，对2024年中国经济走向进行预测，并分析可能出现的变化和挑战。\n4. 政策建议：针对预测结果，提出相关的政策建议，包括宏观调控、产业升级、区域发展等方面。\n5. 结论总结：最后，需要对文章内容进行总结，强调观点并展望未来中国经济的发展前景。',
 'theme': '2024年中国经济走向与运行趋势'}

In [20]:
%%time
allChain = map_chain | articleChain | strParser
allChain.invoke({"theme":query})

CPU times: total: 31.2 ms
Wall time: 33 s


'引言：中国经济在全球的地位和重要性\n中国作为全球第二大经济体，其经济走向与运行趋势不仅影响国内民生福祉，也对全球经济产生重要影响。因此，预测2024年中国经济的走向和运行趋势具有重要意义。\n\n背景：中国经济发展历程、当前形势及挑战\n自改革开放以来，中国经济经历了高速增长阶段，成为世界经济增长的主要引擎之一。然而，随着经济规模的扩大和国际环境的复杂化，中国经济也面临着诸多挑战，包括人口老龄化、环境污染、产业结构调整等。\n\n宏观经济分析：GDP增长、通货膨胀、就业率等指标预测\n根据当前的经济形势和政策走向，预计2024年中国GDP增长率将保持在5-6%之间，通胀率维持在3%左右，就业率保持稳定。然而，这些预测结果可能会受到国际贸易环境、国内消费需求等因素的影响而有所变化。\n\n行业趋势分析：制造业、房地产、金融业、科技产业等发展趋势\n制造业方面，随着供给侧结构性改革的推进和创新驱动发展战略的实施，中国制造业将逐步向高端化、智能化转型。房地产行业将继续坚持“房住不炒”的政策导向，保持平稳健康发展。金融业在防范化解金融风险的同时，将加大对实体经济的支持力度。科技产业将成为推动中国经济高质量发展的重要引擎，人工智能、5G等前沿技术有望取得突破性进展。\n\n政策与改革：政府对经济的调控和改革措施\n2024年，中国政府将继续坚持稳中求进的工作总基调，实施积极的财政政策和稳健的货币政策，保持宏观经济稳定运行。同时，进一步深化改革开放，优化营商环境，激发市场活力和社会创造力。此外，还将继续推进供给侧结构性改革、区域协调发展等重大战略部署。\n\n风险与机遇：外部环境变化、内部结构调整带来的挑战和机会\n外部环境方面，全球经济复苏的不确定性、贸易保护主义抬头等因素可能对中国经济带来一定冲击。内部结构调整方面，人口老龄化、产业结构升级等问题需要妥善应对。然而，这些挑战也蕴含着机遇，如科技创新、消费升级等有望成为新的经济增长点。\n\n结论：对中国经济未来发展的展望和建议\n总体而言，2024年中国经济将继续保持稳定增长，但面临诸多不确定因素和挑战。因此，政府应继续坚持稳中求进的工作总基调，加强宏观调控，深化改革开放，推动经济高质量发展。同时，企业应积极适应市场变化，加大创新投入，提高核心竞争力。个人则需关注就业形势、收入分配等方面的政策调整，合理规划自身发展。'

In [21]:
%%time
allChain = (
    {
        "outline":outlineChain,
        "tips":tipsChain,
        "theme":itemgetter("theme")
    }
    | articleChain
    | strParser
)
allChain.invoke({"theme":query})

CPU times: total: 0 ns
Wall time: 58.7 s


'引言\n中国经济自改革开放以来取得了举世瞩目的成就，已成为全球第二大经济体。然而，随着全球经济格局的变化以及国内经济结构的调整，预测未来经济发展趋势变得尤为重要。本文将分析2024年中国经济的走向与运行趋势，以期为相关决策者和投资者提供参考。\n\n宏观经济环境\n全球经济形势：\n2024年，全球经济仍面临诸多不确定性。一方面，主要经济体之间的贸易摩擦和地缘政治风险可能持续影响全球经济增长；另一方面，新兴市场和发展中国家在全球经济中的比重不断上升，对全球经济格局产生重要影响。在此背景下，中国需要继续推进改革开放，积极参与国际合作，以应对外部环境的变化。\n\n国内政策：\n2024年，中国政府将继续坚持稳中求进的工作总基调，保持宏观政策的连续性和稳定性。货币政策方面，央行将根据经济形势灵活运用多种工具，保持流动性合理充裕；财政政策方面，政府将继续实施积极的财政政策，加大减税降费力度，支持实体经济的发展。此外，政府还将继续推进供给侧结构性改革，推动经济结构优化升级。\n\n人口结构变化：\n中国的人口老龄化问题日益突出，2024年将面临更大的养老压力和劳动力短缺挑战。这将对消费市场、社会保障体系以及科技创新等方面产生深远影响。因此，政府需要采取措施应对人口老龄化带来的社会和经济问题。\n\n产业趋势\n制造业升级：\n随着人工智能、物联网等新兴技术的快速发展，中国制造业正加速向高端制造和服务型制造转型。2024年，智能制造、工业互联网等领域将迎来更多发展机遇。同时，传统制造业也将通过技术改造和转型升级实现提质增效。\n\n服务业发展：\n随着居民消费水平的提高和消费结构的升级，服务业将成为经济增长的重要引擎。2024年，金融、医疗保健、教育培训等现代服务业将继续保持高速增长；与此同时，旅游、文化娱乐等生活性服务业也将迎来新的发展机遇。\n\n科技创新：\n中国政府高度重视科技创新在经济发展中的作用，不断加大研发投入和政策支持力度。2024年，人工智能、5G通信、生物医药等领域将成为科技创新的重点领域，为经济增长提供新动能。\n\n消费市场\n消费升级：\n随着居民收入水平的提高和消费观念的转变，消费者对高品质、个性化产品的需求日益增长。2024年，高端消费品市场将继续保持快速增长；同时，绿色环保产品也将受到更多消费者的青睐。\n\n新兴消费模式：\n电商、移动支付

In [22]:
%%time
allChain = (
    {
        "outline":outlineChain,
        "tips":tipsChain,
        "theme":itemgetter("theme")
    }
    | articlePrompt 
    | model 
    | strParser
)
allChain.invoke({"theme":query})

CPU times: total: 0 ns
Wall time: 48.3 s


'引言：中国经济在全球经济中的重要地位以及近年来取得的成就和面临的挑战\n\n中国作为全球第二大经济体，其经济发展对世界产生了深远的影响。自改革开放以来，中国经济取得了令人瞩目的成就，包括高速经济增长、人民生活水平显著提高、科技创新能力不断增强等。然而，随着全球经济格局的变化和中国自身发展阶段的转变，也面临着诸多挑战，如人口老龄化、环境污染、产业结构调整等。\n\n第一部分：宏观经济环境分析\n\na) 全球经济形势对中国的影响\n\n2024年，全球经济增长预计将保持稳定，但仍然存在不确定性因素，例如贸易摩擦、地缘政治风险和疫情反复等。这些外部因素可能会对中国的出口和投资造成一定影响。然而，中国经济的韧性和内需的潜力有望抵消部分负面影响。\n\nb) 中国国内经济政策调整与改革\n\n近年来，中国政府一直在推进供给侧结构性改革，以提高经济质量和效率。2024年，预计将继续深化改革，包括减税降费、优化营商环境、推动国有企业改革等措施，以激发市场活力和提升竞争力。此外，货币政策将保持稳健中性，财政政策将更加积极有效，为经济增长提供支持。\n\n第二部分：主要行业趋势预测\n\na) 制造业升级与转型\n\n2024年，中国制造业将继续向高端化、智能化、绿色化方向转型升级。随着人工智能、物联网等技术的广泛应用，智能制造将成为制造业发展的主流趋势。同时，新能源、新材料等新兴产业也将得到进一步发展。\n\nb) 消费市场变化与新兴消费模式\n\n随着居民收入水平的提高和消费观念的转变，中国消费市场将呈现多元化、个性化的发展趋势。电子商务、在线教育、医疗健康等领域将继续保持高速增长。此外，共享经济、体验式消费等新兴消费模式也将成为新的经济增长点。\n\nc) 金融业发展与风险防范\n\n2024年，中国金融业将继续深化改革，推动利率市场化、人民币国际化和资本市场开放。同时，加强监管力度，防范金融风险，维护金融市场稳定。此外，金融科技的发展将为金融行业带来更多创新机遇。\n\n第三部分：区域经济发展展望\n\na) 城市群和都市圈建设\n\n2024年，中国将继续推进京津冀协同发展、长三角一体化发展和粤港澳大湾区建设等重大战略，促进区域协调发展。同时，加强中小城市的产业升级和基础设施建设，推动城乡融合发展。\n\nb) 乡村振兴战略的推进\n\n随着农村土地制度改革和农业现代化